**Parameters Used**



In [0]:
 attributes=pd.read_excel("Attributes.xlsx")
 attributes

,Attributes,Explaination,Unnamed: 2
0,gender,Gender,NaN
1,age,Age (respondent),NaN
2,married,Marital status (respondent),NaN
3,children,Number of children,NaN
4,edu,Years of education completed (respondent),NaN
5,hh_totalmembers,Household size,NaN
6,asset_savings,Value of savings (USD),NaN
7,cons_alcohol,Alcohol (USD),NaN
8,cons_tobacco,Tobacco (USD),NaN
9,cons_med_total,Medical expenditure past month (USD),NaN


**Imporing libraries**

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.utils import shuffle
from sklearn.externals import joblib

import math

**Loading training and test data**

In [0]:
df_train = pd.read_csv('train1.csv')
#df_test = pd.read_csv('test.csv')
df_train

,surveyid,survey_date,femaleres,age,married,children,edu,hh_totalmembers,asset_savings,cons_alcohol,cons_tobacco,cons_med_total,cons_ed,cons_social,cons_other,ent_income,med_portion_sickinjured,med_port_sick_child,durable_investment,depressed
0,926,23-Nov-61,1,28.0,1,4,10,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.00000,0
1,747,24-Oct-61,1,23.0,1,3,8,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.00000,1
2,1190,05-Oct-61,1,22.0,1,3,9,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.00000,0
3,1065,23-Sep-61,1,27.0,1,2,10,4.0,49.647648,0.000000,0.000000,4.163996,1.681614,39.771500,44.042267,17.890503,0.000000,0.00,778.11230,0
4,806,12-Sep-61,0,59.0,0,4,10,6.0,0.000000,0.000000,0.000000,0.800768,2.268844,8.087762,74.503502,2.073101,0.333333,0.25,201.00562,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,927,27-Dec-61,1,36.0,0,4,9,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.00000,1
1139,1039,13-Sep-61,1,23.0,1,0,10,2.0,0.000000,20.934376,4.186875,0.000000,0.000000,2.522421,8.488146,0.000000,0.500000,NaN,367.00491,0
1140,428,23-Nov-61,1,28.0,1,5,10,7.0,72.069168,0.000000,0.000000,0.000000,2.922805,10.676913,137.251720,72.936661,0.285714,0.40,490.32327,0
1141,23,05-Oct-61,1,33.0,0,4,9,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.00000,0


In [0]:
# Helper function

def depression(x):
    if(x>0.5):
        return 1
    else:
        return 0

**Data cleaning & preprocessing**

In [0]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        print ("The dataset has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
    
    
missing_df = missing_values_table(df_train);
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
print('\n','%d columns will be deleted.' % len(missing_columns))

df_train = df_train.drop(columns = list(missing_columns))


The dataset has 20 columns.
There are 5 columns that have missing values.

 0 columns will be deleted.


**Normalizing data**

In [0]:
encoder = LabelEncoder()
encoder.fit(df_train['survey_date'])

#encoder.transform(df_train['survey_date'])
df_train['survey_date'] = encoder.transform(df_train['survey_date'])




**Setting up training data**

In [0]:
X = df_train.drop(["depressed","surveyid","survey_date"], axis=1)

# fill missing values with mean column values
imputer = SimpleImputer()
transformed_X = imputer.fit_transform(X)
y = df_train.depressed
input1 = transformed_X[0]

seed = 1
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=test_size, random_state=seed)

**Training neural network classifier using Keras API**

In [0]:
best=0
for _ in range(5):

  X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=test_size, random_state=5)


  # create model
  model = Sequential()
  model.add(Dense(36, input_dim=17, activation='relu'))
  model.add(Dense(36, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #adaptive moment estimation

  # Fit the model
  hist = model.fit(X_train, y_train, epochs=100, batch_size=10)

  scores = model.evaluate(X_test, y_test)
  #scores=model.score(X_test,y_test)
  print(scores[1])
  if scores[1]>best:
      best=scores[1]
      joblib.dump(model , 'PatientHistoryDepressionPrediction.pk1')
      

Epoch 1/100
765/765 [==============================] - 0s 558us/step - loss: 1.3690 - accuracy: 0.7346
Epoch 2/100
765/765 [==============================] - 0s 133us/step - loss: 1.2869 - accuracy: 0.7634
Epoch 3/100
765/765 [==============================] - 0s 129us/step - loss: 0.7437 - accuracy: 0.7582
Epoch 4/100
765/765 [==============================] - 0s 131us/step - loss: 0.6905 - accuracy: 0.7895
Epoch 5/100
765/765 [==============================] - 0s 132us/step - loss: 0.7113 - accuracy: 0.7804
Epoch 6/100
765/765 [==============================] - 0s 133us/step - loss: 0.6629 - accuracy: 0.7843
Epoch 7/100
765/765 [==============================] - 0s 130us/step - loss: 0.5961 - accuracy: 0.7895
Epoch 8/100
765/765 [==============================] - 0s 138us/step - loss: 0.6424 - accuracy: 0.7778
Epoch 9/100
765/765 [==============================] - 0s 147us/step - loss: 0.5850 - accuracy: 0.7987
Epoch 10/100
765/765 [==============================] - 0s 132us/step - l

**Opening saved model**

In [0]:
#model = joblib.load("PatientHistoryDepressionPrediction.pk1")

**Accuracy of model**

In [0]:
scores = model.evaluate(X_test, y_test)          
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

NameError: ignored

**Prediction**

In [0]:

# input1=input1.reshape(1,17)
# predictions=model.predict(input1)
# predictions

array([[0.18336171]], dtype=float32)

In [0]:
y_train.head(20)

779     0
525     0
464     1
600     0
433     0
1098    0
544     1
714     0
178     0
841     0
253     1
1111    0
426     0
602     0
993     0
139     0
446     0
1067    0
634     0
861     0
Name: depressed, dtype: int64

In [0]:
for x in range(20): 
    print(depression(predictions[x]))#,'Actual',y_train[x])

#predictions.head(20)

0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
